# Introduction to Data Science – Relational Databases
*COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/* 

Up to now, we've mainly used flat tables to store and process data. Most structured data in the real world, however, is stored in databases, and specifically in [relational databases](https://en.wikipedia.org/wiki/Relational_database). Other database types have specific advantages, such as performance (NoSQL), are suitable for graphs (graph databases, e.g., Neo4j), or are compatibility with in-memory OO data structures (Object Oriented Databases). Relational databases, and their implementations - Relational Database Management Systems (RDBMS), are still the dominant way to store enterprise data though.

Unlike with APIs or web-scarping, it is unlikely that you will be able to access a public database directly. Databases are powerful, but also can be somewhat difficult to use. It's more likely that an API that you interact with is powerd by a relational database behind the scenes, hiding the complexity.

Also, while we've mainly considered reading data, databases are meant also for writing, increasing the potential for abuse. So the most likely scenario for having read/write access to a database is if the database is managed by you or your organization. 

What's the purpose of a relational database? It separates data into multiple tables to avoid redundancy. Here is a simple example for an online transaction in a flat table that describes the data:

| Product Name | Price | Product Description | Customer Name | Customer Address| 
| ------------- | - | - | - | -| 
| MacBook | 2700 | MacBook Pro 15' | Jack Black | San Francisco |  
| Dongle HDMI | 40 | USB-C to HDMI | Jack Black | San Francisco | 
| Dongle USB-B | 40 | USB-C to USB-B | Jack Black | San Francisco | 

The problem here is that we store the name of the customer multiple times, and, if multiple customers purchase the same item, also all the information about the products. Obviously, that's neither great from a storage efficiency perspective nor from an update perspective. If Jack Black wants to update his address, for example, than we'd have to update all three rows. 

A different approach is to separate this table into multiple, meaningful tables. We'll introduce a Customers, Products and a Transactions table. And we'll introduce keys for each row in this table:

**Customers Table**:

| CID | Customer Name | Customer Address | 
| - | - | - |
| 1 | Jack Black | San Francisco |

**Products Table**:

| PID | Product Name | Price | Product Description |
| - | - | - | - | 
| 1 | MacBook | 2700 | MacBook Pro 15' |
| 2 | Dongle HDMI | 40 | USB-C to HDMI |
| 3 | Dongle USB-B | 40 | USB-C to USB-B |

CID and PID are called **primary keys**, as they uniquely identify the row in the table. 

The Transaction Table now refers to the products and the customers only by their keys, which are called **foreign keys** in this context, as they are the primary keys of a "foreign" table.

**Transactions Table**: 

| TID | PID | CID | 
| - | - | - |
| 1 | 1 | 1 | 
| 2 | 3 | 1 | 
| 3 | 2 | 1 | 

Now, if Jack Black wants to update his address, or if the seller wants to update the product description, we can do this in a single place. 

Of course, we have to do a little more work to regenerate (an interesting subset) of transactions, as we had it before. In this lab we'll learn how to do that and more. 

## Relational Database Management System (RDBMS) Transactions

RDBMS are designed to support **read, write, and update** transactions, in addition to operations that **create tables**, etc. In practice, for a database to remain consistent, these transactions have to guarantee [certain properties](https://en.wikipedia.org/wiki/ACID). We'll largely ignore updates, creating tables, etc. – these are operations that you'll likely need if you build a web-application. For the purpose of our data science class, we'll stick to reading/querying for data, as we want to learn **how to get data out of a database**. You can learn more about databases in CS 6530 or the undergrad version CS 5530.

## SQL - The Structured Query Language


[SQL](https://en.wikipedia.org/wiki/SQL) is a domain specific language to execute the transactions described above. We'll look mainly at queries to read and aggregate data. 

Here is a very simple query:

```SQL
SELECT * FROM products
```

This statement selects and retreives all rows (`*`) from the table `products`.

We can restrict the number of lines to retrieve with a `WHERE` clause:

```SQL
SELECT * FROM products WHERE 'Price' > 100
```

This retrieves all the rows where the price is higher than 100. 

We'll go through more specific SQL statements on real examples. You can learn more about SQL [online](https://www.codecademy.com/learn/learn-sql).


## SQLite

Most database management systems are implemented as client-server systems, i.e., the database is hosted on a dedicated server where multiple users can read and write. Examples are [PostgreSQL](https://www.postgresql.org/), [MySQL](https://www.mysql.com/) or [Oracle](https://www.oracle.com/database/index.html). These database management systems are fast and can handle multiple users withouth causing conflicts. However, they require a separate server and installation, so we'll use [SQLite](https://sqlite.org/), a database that works for individual users and stores the whole database in a single file. SQLite is widely used for single-user cases and will serve our purposes. 

While these different databases have different features and performance characteristics, they all support SQL, so the skills you will learn here transfer widely.

We'll also use the [sqlite python interface](https://docs.python.org/3/library/sqlite3.html).

## Sample Database

This tutorial uses the SQLite sample database found [here](http://www.sqlitetutorial.net/sqlite-sample-database/).

Here is a chart of the database schema: 

![](database_schema.png)

 * `employees` table stores employees data such as employee id, last name, first name, etc. It also has a field named `ReportsTo` to specify who reports to whom.
 * `customers` table stores customers data.
 * `invoices & invoice_items` tables: these two tables store invoice data. The invoices table stores invoice header data and the invoice_items table stores the invoice line items data.
 * `artists` table stores artists data. It is a simple table that contains only artist id and name.
 * `albums` table stores data about a list of tracks. Each album belongs to one artist. However, one artist may have multiple albums.
 * `media_types` table stores media types such as MPEG audio file, ACC audio file, etc.
 * `genres` table stores music types such as rock, jazz, metal, etc.
 * `tracks` table store the data of songs. Each track belongs to one album.
 * `playlists` & `playlist_track` tables: playlists table store data about playlists. Each playlist contains a list of tracks. Each track may belong to multiple playlists. The relationship between the playlists table and tracks table is many-to-many. The playlist_track table is used to reflect this relationship.
 
The diagram above highlights the **primary keys** in each table and the relationships, using **foreign keys** to each other table. Note that `employees` has a self-reference, to capture the reports-to relationship.

## Querying the Table

In [ ]:
import pandas as pd
import sqlite3 as sq

# we connect to the database, which - in the case of sqlite - is a local file
conn = sq.connect("./chinook.db")
# we retreive the "cursor" on the database.
c = conn.cursor()

# now we can execute a SQL statement
c.execute("SELECT * FROM albums")
# and print the first line from the result
print(c.fetchone())

Here, we used a cursor for accessing the data. To retrieve data after executing a SELECT statement, you can either treat the cursor as an iterator, call the cursor’s `fetchone()` method to retrieve a single matching row, or call `fetchall()` to get a list of the matching rows.

This is how you use it as an iterator: 

In [ ]:
for row in c:
    print(row)

And here we retrieve all the data at once:

In [ ]:
c.execute("SELECT * FROM albums")
c.fetchall()

## SQL and Pandas

An alternative approach that fits well into our previous workflows is to use pandas to store the table directly, using the [`read_sql()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql.html) function:

In [ ]:
# this is similar to the read_csv()function
pd.read_sql("""SELECT * FROM albums""", conn).head()

This, of course, works only for reading data, but we'll use it for now. 

## Selective Results

When we **specify individual columns** instead of using the `*`, we get only those columns:

In [ ]:
pd.read_sql("""SELECT Title FROM albums""", conn).head()

Here, we've used head to display a short list, but we can also do that in the SQL query with the `LIMIT` keyword. Of course, in this case, the data will actually be limited: 

In [ ]:
pd.read_sql("""SELECT Title FROM albums LIMIT 3""", conn).head()

We can run more **restrictive queries** with the `WHERE` keyword:

In [ ]:
# note that the triple quotes allow us to break lines
pd.read_sql("""SELECT * FROM albums 
                    WHERE Title = 'Big Ones'""", conn)

Legal operations are: 

 * `=` is.
 * `<>` is not
 * Numerical comparisons: `>, >=, <, <=`
 * `IN` allows you to pass a list of options.
 * `BETWEEN` allows you to define a range.
 * `LIKE` can be used with pattern matching, similar to regular expressions. 
 * `IS NULL` or `IS NOT NULL` test for empty values. 

The `is not` operator:

In [ ]:
pd.read_sql("""SELECT * FROM albums 
                    WHERE Title <> 'Big Ones'""", conn).head(10)

In [ ]:
pd.read_sql("""Select * FROM invoices WHERE Total > 10""", conn)

The `BETWEEN` keyword:

In [ ]:
pd.read_sql("""Select * FROM invoices WHERE Total BETWEEN 15 and 20""", conn)

The `IN` keyword:

In [ ]:
pd.read_sql("""Select * FROM invoices WHERE BillingCity in ('Chicago', 'London', 'Berlin')""", conn)

Expressions with `LIKE`:

In [ ]:
# starts with a C
pd.read_sql("""Select * FROM invoices WHERE BillingCity LIKE 'C%'""", conn)

In [ ]:
# ends with rt
pd.read_sql("""Select * FROM invoices WHERE BillingCity LIKE '%rt'""", conn)

We can use `ORDER BY` to sort the output.

In [ ]:
pd.read_sql("""SELECT * FROM albums 
                    ORDER BY title""", conn)

### Exercise 1: Simple Queries

1. List all the rows in the genres table.
2. List only the genre names (nothing else) in the table.
3. List the genre names ordered by name.
4. List the genre entries with IDs between 13 and 17.
5. List the genre entries that start with an S.
6. List the GenreIds of Rock, Jazz, and Reggae (in one query).

## Referencing and Renaming

Not very exciting on it's own, but we'll want to referene exact names and rename them soon. Here we're using a fully qualified name `tracks.Name` to refer to the name column in the tracks table, and then rename it as `TrackName`.

In [ ]:
pd.read_sql("""SELECT tracks.Name as TrackName FROM tracks;""", conn).head()

## Joining Tables

There are many ways to aggregate and group by with SQL, and we won't be able to cover them in detail. We'll use a simple example for an `INNER JOIN` to resolve a foreign key relationship here.

We want to create a long table that contains both track titles and for each track the album title. Let's take a looks at the tracks table:

In [ ]:
pd.read_sql("""SELECT * FROM tracks;""", conn).head()

And the albums table:

In [ ]:
pd.read_sql("""SELECT * FROM albums;""", conn).head(5)

We can see that AlbumID appears in both tables. This is the relationship we care about:

![](albums_tracks_tables.jpg)

An INNER JOIN simply merges tables based on a primary key / foreign key relationship. 

Here is an example:

In [ ]:
pd.read_sql("""SELECT
 trackid,
 tracks.name as Track,
 albums.title as Album
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid""", conn).head(30)

Let's take this statement apart:

```SQL
SELECT
 trackid,
 tracks.name as Track,
 albums.title as Album
```
Here, we chose what to select, and we also rename the columns for presentation purposes with the `as` statement. 

```SQL
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid;
```

This is where the magic happens: we say that the tracks and albums table should be joined on the `albums.albumid = tracks.albumid` relationship.

Using INNER JOIN, you can construct flat tables out of databases that you then can use in the data science process. SQL is of course much more powerful, you can do mathematical operations, group-by/aggregates, etc. However, we can do most of this on a dataframe also, but doing a join like that would be rather painful without the INNER JOIN SQL statement.

Next, we want to also add the artist name. We can just append another `INNER JOIN` statement.

In [ ]:
pd.read_sql("""SELECT
 tracks.name as Track,
 albums.title as Album,
 artists.name as Artist 
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid
INNER JOIN artists ON artists.artistid = albums.artistid""", conn).head(40)

Now we can combine that with a `WHERE` condition to get a more specific result. Here we're looking at all the Rolling Stones albums in this dataset. 

In [ ]:
pd.read_sql("""SELECT
 tracks.name as Track,
 albums.title as Album,
 artists.name as Artist 
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid
INNER JOIN artists ON artists.artistid = albums.artistid
WHERE artists.name = 'The Rolling Stones'""", conn).head(40)

## Group By

We've seen group by before for dataframes, and we could just use pandas' group by since we're using a dataframe anyways. However, it's useful to understand how group by works in SQL as it can be much more efficient to just get the right data out in the first place. 

In [ ]:
pd.read_sql("""SELECT
 albumid,
 COUNT(trackid) as '# Tracks'
FROM
 tracks
GROUP BY
 albumid;""", conn).head()

These are accurate counts, but now we want to combined them with album titles. We first do the INNER JOIN and then the GROUP BY. 

In [ ]:
pd.read_sql("""SELECT
 tracks.albumid,
 albums.title as Album,
 COUNT(tracks.trackid)
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid
GROUP BY
 tracks.albumid""", conn).head(30)

We can combine this with filters, using `HAVING COUNT`.

In [ ]:
pd.read_sql("""SELECT
 tracks.albumid,
 albums.title as Album,
 COUNT(tracks.trackid)
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid
GROUP BY
 tracks.albumid
HAVING COUNT(tracks.trackid) > 15;""", conn).head(30)

Here are several other aggregation functions, `AVG, MAX` and `SUM`. We also do some simple math to convert miliseconds into minutes for the average: 

In [ ]:
pd.read_sql("""SELECT
 tracks.albumid,
 albums.title as Album,
 COUNT(tracks.trackid),
 AVG(tracks.Milliseconds)/1000/60 as 'Average(minutes)',
 MAX(tracks.Milliseconds),
 SUM(tracks.Milliseconds)
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid
GROUP BY
 tracks.albumid
ORDER BY 
  AVG(tracks.Milliseconds) DESC""", conn).head(30)

## Exercise 2: Joining

1. Create a table that contains track names, genre name and genre ID for each track. Hint: the table is sorted by genres, look at the tail of the dataframe to make sure it works correctly.
2. Create a table that contains the counts of tracks in a genre by using the GenreID.
3. Create a table that contains the genre name and the count of tracks in that genre.
4. Sort the previous table by the count. Which are the biggest genres? Hint: the DESC keyword can be added at the end of the sorting expression.

## Security

We can use python variables to specify the columns. However, you typically shouldn't trust the content of the variables, especially in a user facing system. Imagine, you read in which attribute to query for from a website field where a user can specify a name. The user could use this to attack your SQL server using [SQL Injection](https://en.wikipedia.org/wiki/SQL_injection). For example, for this statement:

In [ ]:
title = "Big Ones"
# executescript runs multiple sql statements. 
# This isn't very helpful for quering, but can be great for other operations.
c.executescript("""SELECT * FROM albums WHERE Title = '""" + title + "'")

Let's look at the tables in the database:

In [ ]:
pd.read_sql("""SELECT name FROM sqlite_master
            WHERE type='table'
            ORDER BY name;
            """, conn)

Now, if we were to read `title` from a user, and adversarial user could write: 

```SQL
"a';DROP TABLE invoice_items;"
```

This selects everything from table "a", then concludes the SQL command with `;`, and then executed the next SQL command `DROP TABLE invoice_items`, which – you guessed it – deletes the table invoice_items.

In [ ]:
title = "a';DROP TABLE invoice_items;"
print("""SELECT * FROM albums WHERE Title = '""" + title + "'")
c.executescript("""SELECT * FROM albums WHERE Title = '""" + title + "'")

This throws an error, but the table is gone:

In [ ]:
pd.read_sql("""SELECT name FROM sqlite_master
            WHERE type='table'
            ORDER BY name;
            """, conn)

Instead, you should use a ? for substitution, and pass in a tuple. This will make sure that no additional statements will be executed. 

In [ ]:
title = ("a';DROP TABLE invoice_items;",)
# this will result in an error
c.executescript('SELECT * FROM albums WHERE title=?', title)

Here is a working example with safe code:

In [ ]:
title = ("Big Ones",)
# this will result in an error
c.execute('SELECT * FROM albums WHERE title=?', title)
c.fetchone()

Of course that's not all to say about making your code secure, but it's a start! 

![](exploits_of_a_mom.png)